# Add group to OME-Zarr V0.4 (PyMIF 0.2.2)

In [ ]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import time
import glob

In [ ]:
# --- Set input data path ---
main_path = "../"
dataset_name = "E_for_Nicola"

outpath = "../"
outdataset_name = "cephalopods_ome-zarr_V04_with_groups_and_labels"

# main_path = "/g/mif/common/Data/Z_from_LCS_24-07-16/Coll_Montse/MouseEmbryo_micemetyleneBlue/2022-08-09_132304/processed"
# dataset_name = "20221108-162726_Task_2_LM_FullRes_C"

# main_path = "/mif-users/Users/Arias_Adrian/scans_CSL_P21/2400_31573_s"
# dataset_name = "20250523-105843_Task_1_stitching_1_C"

data_path = f"{main_path}/{dataset_name}"

output_zarr_path = f"{outpath}/{outdataset_name}.zarr"

In [ ]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: None = zarr native chunks)
zarr_dataset = mm.ZarrManager(path=output_zarr_path, mode="a")

In [ ]:
print(zarr_dataset.data)
print(zarr_dataset.groups)
print(zarr_dataset.labels)


In [ ]:
# zarr_dataset.visualize_zarr()

## Now add a new group e.g. processed and append a small part of the image after some processing

In [ ]:
metadata = {
    "axes" : "tczyx",
    "size" : [tuple([s[0],1,s[2],s[3],s[4]]) for s in zarr_dataset.metadata["size"]],
    "chunksize" : zarr_dataset.metadata["chunksize"],
    "scales" : zarr_dataset.metadata["scales"],
    "time_increment" : zarr_dataset.metadata["time_increment"],
    "time_increment_unit" : zarr_dataset.metadata["time_increment_unit"],
    "units" : zarr_dataset.metadata["units"],
    "channel_names" : ["GFP_processed"],
    "channel_colors" : [16711935],
}

zarr_dataset.create_empty_group(group_name="processed",
                                     metadata=metadata,
                                     is_label=False)

In [ ]:
from skimage.filters import gaussian
array = zarr_dataset.data[0][0,1,...].compute()
array = gaussian(array, sigma=1, preserve_range=True)
array = array[None,None,...]
print(array.shape)

In [ ]:
import numpy as np
shape = zarr_dataset.data[0].shape

zarr_dataset.write_image_region(
    data = np.array(array),
    t = slice(0,1),
    c = slice(0,1),
    z = slice(0,shape[2]),
    y = slice(0,shape[3]),
    x = slice(0,shape[4]),
    level = 0,
    group = "processed",
)

In [ ]:
viewer = napari.Viewer()
zarr_dataset = mm.ZarrManager(path=output_zarr_path, mode="r")
# zarr_dataset.visualize_zarr(viewer=viewer)

In [ ]:
print(zarr_dataset.data)
print(zarr_dataset.groups)
print(zarr_dataset.labels)